# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Открываем файл с данными и изучаем общую информацию

In [15]:
# Подключаем библиотеку Pandas 
# Подключаем библиотеку с функцией лемматизации на русском языке. Для поиска слов записанных в разной форме

In [16]:

import pandas as pd
from pymystem3 import Mystem 
m = Mystem()

data = pd.read_csv('/datasets/data.csv')

<h3>Исследуем загруженные данные </h3>

In [17]:
# Узнаем количество строк и столбцов в датасете
data.shape

(21525, 12)

In [18]:
# Просмотрим общую информацию о данных. Названия колонок, количество строк в конкретной колонке, типы данных.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [19]:
# Сделаем визуальный обзор верхнего кадра датасета, выведем первые 20 строк. Определим категориальные и количественные колонки. 
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [20]:
#  Проверим строки NaN в датасете
data[pd.isnull(data['days_employed'])]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [21]:
# столбец childeren нобходим для исследования, посчитем поличество категорий в колонке
data['children'].value_counts() 

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [22]:
# посчитаем категории в колонке education
data['education'].value_counts() 

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [23]:
# посчитаем категории в колонке family_status
data['family_status'].value_counts() 

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [24]:
# посчитаем категории в колонке purpose
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

**Вывод**

В датасте 21525 строк, 12 колонок
<font color='gray'><p>
<li>Значения в столбце children выражены типом int и, с точки зрения статистики, являются категориальными.</li> 
<li>Значения в столбце days_employed  выражены типом float и, с точки зрения статистики, являются количественными.</li> 
<li>Значения в столбце dob_years  выражены типом int и, с точки зрения статистики, являются количественными.</li> 
<li>Значения в столбце education выражены типом str и, с точки зрения статистики, являются категориальными.</li>  
<li>Значения в столбце education_id выражены типом int и, с точки зрения статистики, являются категориальными.</li>  
<li>Значения в столбце family_status выражены типом str и, с точки зрения статистики, являются категориальными.</li>  
<li>Значения в столбце family_status_id выражены типом int и, с точки зрения статистики, являются категориальными.</li>  
<li>Значения в столбце gender выражены типом str и, с точки зрения статистики, являются категориальными.</li>  
<li>Значения в столбце income_type выражены типом str и, с точки зрения статистики, являются категориальными.</li>  
<li>Значения в столбце debt выражены типом bool и, с точки зрения статистики, являются категориальными.</li>  
<li>Значения в столбце total_income выражены типом float и, с точки зрения статистики, являются количественными.</li>  
<li>Значения в столбце purpose выражены типом str и, с точки зрения статистики, являются категориальными.</li></p></font>
<br>

<p>
<li>С помщью метода info() видим что в датафрейме есть пустые значения</li> 
<li>В колонке  children есть отрицательныое заначение и категория с количеством 20 детей (скорее всего ошибка в заполнении данных) </li> 
<li>В имени категорий колонки education используется верхний регистр. Это создает дубли и проблемы при категоризации.</li>
<li>В колонке family_status в наименовании категорий есть верхний регистр, нужно привести к нижнему</li>
<li>В колонке purpose  есть похожие категории по смыслу, исправим с помощью лемматизации</li>
<li>В колонке days_employed имеются отрицательные значения. Количестсто дней трудового стажа не может буть отрицательным, возможно ошбика в заполнении</li>
<li>Есть взаимосвязь days_employed и total_income по значению NaN. Отсутсвие стажа связано с отсутсвием дохода</li>
</p>


##  Предобработка данных

### Обработка пропусков

In [25]:
# Определяю количество значений NaN в столбцах с пропусками

In [26]:
data.isna().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

<font color='gray'> В колонке days_employed 2174 пропуска со заначением NaN <br> 
В колонке total_income  2174 пропуска со значением NaN</font>

In [27]:
# Сгруппируем данные total_income по категориями income_type и змеменим пропуски на медианные с помощью transform
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))

In [28]:
# Сгруппируем данные days_employed  по категориям dob_years и змеменим пропуски на медианные с помощью transform

data['days_employed'] = data['days_employed'].apply(lambda x: x/24 if x>0  else abs(x))

data['days_employed'] = data['days_employed'].fillna(data.groupby('dob_years')['days_employed'].transform('median'))

In [29]:
data.isna().sum() 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [30]:
data.head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [31]:
# Уберем отрицательные значения в колонке children
data['children'] = abs(data['children'])

In [32]:
# Проверим наличие отрицательных значений в колонке children
data['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [33]:
# Заминим в колонке children категорию количество детей 20 на 2. Скорее всего при заполнении данных опечатались.
data.loc[data['children'] == 20, 'children'] = 2


In [34]:
# Проверим замену категории 20 на 2
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [35]:
# Убирем отрицательные значения в датасете в столбце days_employed
data['days_employed'] = abs(data['days_employed'])

In [36]:
# Проверим клонку gender на уникальные значения и количество упоминаний 
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [37]:
# Проверим что представляет из себя строка со сзначением XNA
data.loc[data.loc[:,'gender'] == 'XNA']  

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Изучая строку сложно сразу определить к какому полу принадлежит заемщик. Зацепкой для определения пола может служить показатель месячного дохода. В данной строке доход достаточно высокий. Проверим средние доходы по половому признаку. Уточним чей средний доход ближе к доходу в строке с XNA

In [38]:
# Вычисляем средний доход у мужчин
data.loc[data.loc[:,'gender'] == 'M']['total_income'].mean() 

189005.6628540888

In [39]:
# Вычисляем средний доход у женщин
data.loc[data.loc[:,'gender'] == 'F']['total_income'].mean() 

153048.46404320325

Средний доход мужчин ближе к значению дохода в строке с XNA. Заменим некорректное значение на M. Даже если предположение неверно, то оно не окажет значительного влияния на исследование поставленных вопросов.

In [40]:
# Заменим XNA  на значение M
data.loc[data['gender'] == 'XNA', 'gender'] = 'M'

In [41]:
# Проверим замену XNA на M
data['gender'].value_counts()

F    14236
M     7289
Name: gender, dtype: int64

In [42]:
# Проверим уникальные значения с клонке с возрастом заемщиков
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В колонке dob_years есть значения 0, заменим на медианное значение

In [43]:
data['dob_years'].median()

42.0

In [44]:
data.loc[data['dob_years'] == 0, 'dob_years'] = 42

In [45]:
# Проверим замену
data['dob_years'].value_counts()

42    698
35    617
40    609
41    607
34    603
38    598
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [46]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.479721,4629.994398,43.490453,0.817236,0.972544,0.080883,1.652253e+05
std,0.755528,5317.592062,12.218595,0.548138,1.420324,0.272661,9.804367e+04
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1007.368820,34.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,2170.569747,42.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,5489.396756,53.000000,1.000000,1.000000,0.000000,1.955499e+05
max,5.000000,18388.949901,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [47]:
# В колонке days_employed видим большой показатель среднего и максимального стажа. 
# Средний стаж по выходу на пенсию в России составляет 8176.5 дней
# Проверим в days_employed показатели выше среднего стажа по выходу на пенсию
data.loc[data.loc[:,'days_employed'] >= 8176.5]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
12,0,14902.851210,65,среднее,1,гражданский брак,1,M,пенсионер,0,118514.486412,сыграть свадьбу
18,0,16678.380705,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
19,0,10038.818549,48,СРЕДНЕЕ,1,в разводе,3,F,сотрудник,0,242831.107982,на покупку своего автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,14121.036100,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,16104.071420,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,15090.043922,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,15583.154618,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


Некорректные данные в количественных переменных заполняют характерными значениями. Значения характеризующие состояние набора данных для проведения исследования. Обчно используют медиану или среднее значение. Сделаем замену слишком больших данных о стаже на медианное значение.

In [48]:
data.loc[data['days_employed'] >= 8176.5, 'days_employed'] = data['days_employed'].median()
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,2170.569747,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,2170.569747,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,2170.569747,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [49]:
# Проверим теперь средний и максимальный показатель days_employed
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.479721,2104.274736,43.490453,0.817236,0.972544,0.080883,1.652253e+05
std,0.755528,1525.620627,12.218595,0.548138,1.420324,0.272661,9.804367e+04
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1007.368820,34.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,2170.569747,42.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,2461.839054,53.000000,1.000000,1.000000,0.000000,1.955499e+05
max,5.000000,8172.406882,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В колонках days_employed, total_income имелись пустые значения. Отстутвие количества дней трудового стажа соответсвует отсутсвию дохода. Поэтому пустые значения можно заменить на 0. Использовали метод .fillna()

### Замена типа данных

In [337]:
# Приведем все нижнему к регистру.
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [338]:
data['education'] = data['education'].str.lower()

In [339]:
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [340]:
# Изменим тип данных с float на int методом .astype    — Этим методом переводят значения в нужный тип.  
data['days_employed'] = data['days_employed'].astype(int)
data['total_income'] = data['total_income'].astype(int)

In [341]:
# Проверим изменение типов данных 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
days_employed       21525 non-null float64
dtypes: float64(1), int64(7), object(5)
memory usage: 2.1+ MB


**Вывод**

<li>Привели к нижнему регистру наименования категорий в колонках education и family_status, избавились от дублей категорий</li>
<li>В колонках со стажем и размером заработной платы преобразовали тип данных с float на int. Использовали метод .astype</li>


### Обработка дубликатов

In [342]:
# Поиск явных дубликатов
data.duplicated().sum()

72

In [343]:
# Посмотрим датафрейм с дубликатами
duplicated_data = data[data.duplicated()].head(20) 
display(duplicated_data) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed
2849,0,1574,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи,-1574.202821
3290,0,365213,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу,365213.306266
4182,1,1574,34,высшее,0,гражданский брак,1,F,сотрудник,0,142594,свадьба,-1574.202821
4851,0,365213,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба,365213.306266
5557,0,365213,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу,365213.306266
6312,0,1574,30,среднее,1,женат / замужем,0,M,сотрудник,0,142594,строительство жилой недвижимости,-1574.202821
7808,0,365213,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы,365213.306266
7921,0,365213,64,высшее,0,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы,365213.306266
7938,0,365213,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы,365213.306266
8583,0,365213,58,высшее,0,не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование,365213.306266


In [344]:
# Удаление явных дубликатов
data = data.drop_duplicates().reset_index()

In [345]:
# Проверим удлаились ли дубликаты
data.duplicated().sum()

0

In [346]:
# Методом unique() проверим неявный дубли

In [347]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [348]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [349]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [350]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [351]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Вывод**

<li>Выявили и удалили явные дубли таблице.</li>
<li>Выяывили неявные дубли в колонке purpose, к примеру 'высшее образование' и 'получение высшего образования' являются смысловыми дублями. Для обработки потребуется лемматизация</li>


### Лемматизация

In [352]:
# Функция для лемматизации категорий колонки purpose. 
# Приведем слова строк к словарной форме и добавим соответсвущие значения в выбранные категории.
def purpose_lemma(value):
    lemmas = m.lemmatize(value)
    for word in lemmas:
        try:
            if 'свадьба' in word:
                return 'свадьба'
        
            if 'недвижимость' in word:
                return 'недвижимость'
        
            if 'жилье' in word:
                return 'недвижимость'
        
            if 'автомобиль' in word:
                return 'автомобиль'
        
            if 'образование' in word:
                return 'образование'

        except:
            print('Error in lemmatization')

In [353]:
# новая колонка с распределенными данными
data['grouped_purpose'] = data['purpose'].apply(purpose_lemma)

In [354]:
# проверим какие категории теперь у нас имеются
data['grouped_purpose'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2323
Name: grouped_purpose, dtype: int64

**Вывод**

<li>Лемматизацией удалось избавится от неявных дублей категорий. Выявили основные четыре категории в колонке purpose</li>

### Категоризация данных

In [355]:
#  Рассчитаем медианау колонки total_income
data['total_income'].median() 


142594.0

In [356]:
# Сегметируем численные данные на категории с помощью функции cut. Полученные категории поместим в колонку grouped_total_income, которую в свою очередь добавим в датафрейм.
data['grouped_total_income'] = pd.cut(data['total_income'], bins=[0, 54000, 135781, 200000, 2300000], include_lowest=True, labels=['низкий доход', 'средний доход', 'доход выше среднего','высокий доход'])

In [357]:
data['grouped_total_income'].value_counts()

средний доход          8484
доход выше среднего    7376
высокий доход          5067
низкий доход            526
Name: grouped_total_income, dtype: int64

In [358]:
data.head()

,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed,grouped_purpose,grouped_total_income
0,0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,-8437.673028,недвижимость,высокий доход
1,1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,-4024.803754,автомобиль,средний доход
2,2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,-5623.422610,недвижимость,доход выше среднего
3,3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,-4124.747207,образование,высокий доход
4,4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,340266.072047,свадьба,доход выше среднего


**Вывод**

1. Использовали медиану колонки total_income для формирования категорий уровня дохода заемщиков
2. Взяли данные по средней заработной плате в России за 2021 г. для формирования категорий. Числовые данные разделили на несколько сегментов (категорий) для дальнейшего анализа. Использовали для этого функцию cut
3. Создали новую колонку grouped_total_income в дадафрейме и поместили в нее категории уровня дохода


In [381]:
# Создадим категории бездетные, малодетные, многодетные 
def group_children(children):
    if children == 0:
        return 'бездетные'
    if 0 < children < 3:
        return 'малодетные'
    if  children > 2:
        return 'многодетные'
    

In [ ]:
data['group_children'] = data['children'].apply(group_children)

In [374]:
data['group_children'].value_counts()

бездетные      14090
малодетные      6983
многодетные      380
Name: group_children, dtype: int64

In [382]:
# Создадим категории бездетные, имеют детей
def group_children_two(children):
    if children == 0:
        return 'Нет детей'
    else:
        return 'Есть дети'

In [383]:
data['group_children_two'] = data['children'].apply(group_children_two)

In [384]:
data['group_children_two'].value_counts()

Нет детей    14090
Есть дети     7363
Name: group_children_two, dtype: int64

##  Отвечаем на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [367]:
# С помощью pivot_table посмотрим сводную таблицу. Посмотрим взаимосвязь колонок children и debt
pd.pivot_table(data, index='children', values='debt', aggfunc = ['count', 'sum', 'mean'])

,count,sum,mean
,debt,debt,debt
children,,,
0,14090,1063,0.075444
1,4855,445,0.091658
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


In [386]:
pd.pivot_table(data, index='group_children', values='debt', aggfunc = ['count', 'sum', 'mean'])

,count,sum,mean
,debt,debt,debt
group_children,,,
бездетные,14090,1063,0.075444
малодетные,6983,647,0.092654
многодетные,380,31,0.081579


In [387]:
pd.pivot_table(data, index='group_children_two', values='debt', aggfunc = ['count', 'sum', 'mean'])

,count,sum,mean
,debt,debt,debt
group_children_two,,,
Есть дети,7363,678,0.092082
Нет детей,14090,1063,0.075444


**Вывод**

1. Заемщики с пятью детьми не имеют задолженностей по кредитам. 
2. Малодетные возращают кредиты хуже бездетных и многодетных.
2. Бездетные возвращают кридиты лучше других категорий.


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [360]:
pd.pivot_table(data, index='family_status', values='debt').sort_values(by='debt', ascending=True)

,debt
family_status,
вдовец / вдова,0.065693
в разводе,0.071130
женат / замужем,0.075452
гражданский брак,0.093494
не женат / не замужем,0.097509


**Вывод**

Семейное положение влиятет на возврат кредита заемщиками. Заемщики имевшие или имеющие узаконенные отношения чаще возвращают кредиты.  Заемщики состоящие в гражданском браке и заемщики ведущие одинокий образ жизни кредиты возвращают реже. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [361]:
pd.pivot_table(data, index='grouped_total_income', values='debt').sort_values(by='debt', ascending=True)

,debt
grouped_total_income,
низкий доход,0.064639
высокий доход,0.070653
средний доход,0.083451
доход выше среднего,0.086903


**Вывод**

Возвращаемость кредиов выше у заемщиков с высоким доходом. 


- Как разные цели кредита влияют на его возврат в срок?

In [362]:
pd.pivot_table(data, index='grouped_purpose', values='debt').sort_values(by='debt', ascending=True)

,debt
grouped_purpose,
недвижимость,0.072334
свадьба,0.080069
образование,0.092200
автомобиль,0.093590


**Вывод**

Заемщики с целью приобретения недвижимости возвращают кредиты чаще. Возможно это связано с тем, что недвижимость на весь период займа находится в залоге у банка. В случае невозврата кредита можно потеярять недвижимость. Во стальных случаеях потери менее ощутимы.

## Общий вывод

1. Замечено влияние количества детей у заемщика на возвращаемость кредита. Чем больше детей, тем меньше вероятность возврата кредита.
2. Семейное положение заемщика также влияет на возращаемость кредита. Заемщики имещие неофрмленный брак и заемщики ведущие одинокий образ жизни кредиты возвращают реже. 
3. Заемщики с целью на приобретение недвижимости возвращают кредиты чаще заемщиков с прочими целями.
4. Заемщики с более высоким доходом возращают кредиты чаше. Заемщики со срденим доходом переоценивают свои возможности, вовращают кредиты реже.